# 연관성 분석
## 1. 라이브러리 등록
> TransactionEoncoder는 각기 다른 길이의 리스트를 같은 길이의 행렬로 만들고 객체가 존재하는 경우 TRUE를 존재하지 않는 경우 false값을 저장한다.

>apriori는 연관성 분석을 위해 사용된다.

>association_rules는 신뢰도 및 향상도를 구하기 위해 사용된다.

## mlxtend 패키지 설치
```
pip install mlxtend
```

In [2]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

## 2. 데이터 만들기
>이론에서 했던 예제를 만들어 봤다. type는 현재 객체의 형식을 확인할 수 있다. apriori는 dataframe이라는 형식을 받아 처리하기 때문에 현재 어떠한 형식인지 확인하는 것은 중요하다.

In [3]:
dataset = [
    ['과자','라면','썬크림','달걀'],
    ['과자','라면','맥주'],
    ['라면','소주','달걀'],
    ['소주','맥주','과자'],
    ['라면','파','달걀']
]

type(dataset)

list

## 3. 데이터 변환
>TransactionEncoder의 fit은 위의 dataset에서 각기 다른 길이의 row들을 모든 경우의 수로 표현하고 같은 길이로 변경한다. 현재 모든 경우는 	[과자,	달걀,	라면, 맥주,	 소주, 썬크림, 파] 총 7가지 이므로 7개의 column과 5개의 row로 이루어진 행렬을 만들게 된다.

>transform은 위에서 만들어진 행렬에서 column명과 같은 값이면 true로 아니면 false값을 반환한다. user0는 [과자, 라면, 썬크림, 달걀]을 구매했으며 현재 column 명은 	[과자,	달걀,	라면, 맥주,	 소주, 썬크림, 파]이다. 과자는 첫 번째 column에 존재함으로 true를 반환하게 되며 라면은 세 번째 column에 있음으로 이 위치에 true를 반환하게 된다. 하지만 column 명 네 번째의 맥주는 구매 내역에 없음으로 false를 반환하게 된다. 이렇게 구매한 상품과 아닌 상품을 true, false로 변환하는 것이 transform이다.


In [4]:
te = TransactionEncoder()
te_fit = te.fit(dataset)
te_fit

TransactionEncoder()

In [5]:
array_matrix = te_fit.transform(dataset)
array_matrix

array([[ True,  True,  True, False, False,  True, False],
       [ True, False,  True,  True, False, False, False],
       [False,  True,  True, False,  True, False, False],
       [ True, False, False,  True,  True, False, False],
       [False,  True,  True, False, False, False,  True]])

In [6]:
te.columns_

['과자', '달걀', '라면', '맥주', '소주', '썬크림', '파']

In [7]:
df = pd.DataFrame(array_matrix, columns=te.columns_)
df

,과자,달걀,라면,맥주,소주,썬크림,파
0,True,True,True,False,False,True,False
1,True,False,True,True,False,False,False
2,False,True,True,False,True,False,False
3,True,False,False,True,True,False,False
4,False,True,True,False,False,False,True


>이렇게 입력된 객체는 array형으로 apriori에서 진행할 수 없다. dataframe으로 형식을 변환하기 위해 pandas의 DataFrame을 사용한다. TransactionEncoder의 columns_는 판매된 모든 제품명을 갖게된다. DataFrame 변환 시 columns 옵션을 이용하면 헤더에 이러한 이름을 적용할 수 있다.

## 4. 데이터 분석
>min_support는 최소 지지도를 지정한다. 너무 낮은 정보를 지정하면 분석 시 시간이 오래 걸리며 낮은 지지도는 그만큼 판매가 덜한 상품이므로 굳이 분석을 할 필요가 없게 된다. 하지만 상품이 많은 경우 전체적으로 지지도가 떨어짐으로 이럴때에는 지지도를 낮춰서 사용하기도 한다. 따라서 상황에 따라 이 최소 지지도를 적당히 조절하여 보다 나은 분석 결과를 도출해 낼 수 있다.

>use_colnames는 실행 결과에 column 이름을 출력할 지 여부를 결정한다.

In [8]:
frequent_itemsets = apriori(df, min_support=0.4, use_colnames=True)
print(frequent_itemsets)

   support  itemsets
0      0.6      (과자)
1      0.6      (달걀)
2      0.8      (라면)
3      0.4      (맥주)
4      0.4      (소주)
5      0.4  (라면, 과자)
6      0.4  (맥주, 과자)
7      0.6  (라면, 달걀)


>dataframe에서 특정 column의 이름을 입력하면 그 column의 정보를 얻을 수 있다.
첫 번째 줄에서 itemsets의 정보를 얻어 itemsets이라는 변수에 저장했다 그리고 결과를 출력한 모습을 볼 수 있다.

>이렇게 얻은 정보에서 lambda 함수를 이용하여 각 객체의 수를 얻을 수 있다. apply는 DataFrame에 함수를 적용하기 위해 사용된다. 세 번째 줄은 객체정보가 x에 입력되면 :(콜론) 뒤의 len(x)를 실행하는 내용으로 객체의 길이를 구하게 되는 것이며 그 결과를 출력하게 된다.

In [9]:
itemsets = frequent_itemsets['itemsets']
print(itemsets)

0        (과자)
1        (달걀)
2        (라면)
3        (맥주)
4        (소주)
5    (라면, 과자)
6    (맥주, 과자)
7    (라면, 달걀)
Name: itemsets, dtype: object


In [10]:
itemsets_x = itemsets.apply(lambda x : len(x))
print(itemsets_x)

0    1
1    1
2    1
3    1
4    1
5    2
6    2
7    2
Name: itemsets, dtype: int64


>DataFrame에 column 이름을 지정하면 별도의 column을 지정할 수 있다. 첫 번째 코드는 기존 객체 정보에서 각 객체별 개수를 구하여 length column에 저장하는 코드이다. 이렇게 해서 결과를 출력해 보면 기존 column에 length column 정보가 추가된 것을 볼 수 있다.

>세 번째 줄은 객체의 수가 2개 이상인 경우만 별도로 출력하는 코드이다. 이렇게 필터링하는 이유는 1개의 객체가 연관 검색에 아무런 의미도 없기 때문에 제외시킨 것이다.

>아래 코드에서 다음과 같이 수정하게 되면 에러가 발생한다.
* frequent_itemsets = frequent_itemsets[frequent_itemsets['length']>=2]

> 이는 달걀과 라면의 지지도를 알 수 없음으로 발생되는 에러이다.


In [11]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x:len(x))
print(frequent_itemsets)

   support  itemsets  length
0      0.6      (과자)       1
1      0.6      (달걀)       1
2      0.8      (라면)       1
3      0.4      (맥주)       1
4      0.4      (소주)       1
5      0.4  (라면, 과자)       2
6      0.4  (맥주, 과자)       2
7      0.6  (라면, 달걀)       2


In [12]:
frequent_itemsets[frequent_itemsets['length'] >= 2]

,support,itemsets,length
5,0.4,"(라면, 과자)",2
6,0.4,"(맥주, 과자)",2
7,0.6,"(라면, 달걀)",2


>최종적으로 association_rules를 이용하여 신뢰도와 향상도를 확인할 수 있다.

>metric은 확인하고자 하는 정보를 입력하면 된다. 현재는 confidence를 입력하여 신뢰도를 기준으로 결과를 확인하게 된다. 향상도를 확인할 경우 lift를 입력하면 된다.

>min_threshold는 metric의 최소 정보이다.

>영향력(lererage)은 두 가지가 모두 나타나는 상황에서 각각의 독립적으로 나타나는 비율을 뺀 것으로 범위는 -1~1까지이며 0이하일 경우 독립적이라고 볼 수 있고 0 초과인 경우 연관성이 있다고 볼 수 있다. 다음과 같은 수식으로 영향력을 표현할 수 있다.
* Leverage(X=>Y) = Support(X,Y) - (Support(X) * Support(Y)) = P(X,Y) - (P(X) * P(Y))

>확신성(confidence)은 명확히 어떤한 값을 갖는지는 모르겠지만 향상도가 1보다 큰 경우 확신성 역시 1보다 큰값을 갖는다. 확신성의 공식을 풀어 보면 결과가 존재하지 않는 경우에서 원인에 따른 결과가 발생하지 않는 경우를 나눈 것으로 수가 클 수록 확신성이 높다고 판단된다. 다음과 같은 수식으로 확신성을 표현할 수 있다.
* Conviction(X=>Y) = ( 1 - Support(Y)) / (1 - Confidence(X=>Y))

In [13]:
ar = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)
ar.sort_values("lift", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2,(맥주),(과자),0.4,0.6,0.4,1.000000,1.666667,0.16,inf,0.666667
3,(과자),(맥주),0.6,0.4,0.4,0.666667,1.666667,0.16,1.8,1.000000
5,(달걀),(라면),0.6,0.8,0.6,1.000000,1.250000,0.12,inf,0.500000
4,(라면),(달걀),0.8,0.6,0.6,0.750000,1.250000,0.12,1.6,1.000000
0,(라면),(과자),0.8,0.6,0.4,0.500000,0.833333,-0.08,0.8,-0.500000
1,(과자),(라면),0.6,0.8,0.4,0.666667,0.833333,-0.08,0.6,-0.333333
